In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [3]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


### Q1. How many columns are in the January dataset?

In [4]:
len(df_jan.columns)

19

### Q2. What is the standard deviation of the duration of January rides in minutes?

In [5]:
df_jan['duration'] = df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']
df_jan['duration'] = df_jan['duration'].apply(lambda td: td.total_seconds() / 60)
df_jan['duration'].std()

42.594351241920904

### Q3. What is the fraction of data left after removing outliers?

In [6]:
((df_jan.duration >= 1) & (df_jan.duration <= 60)).mean()

0.9812202822125979

In [7]:
df_jan = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]

### Q3. What is the dimensionality of the pickup columns after dimensionality reduction?

In [9]:
categorical = ['PULocationID', 'DOLocationID']

df_jan[categorical] = df_jan[categorical].astype(str)

X_dict = df_jan[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(X_dict)

X_train.shape

(3009173, 515)

### Q4. Train a Linear Regression model and obtain RMSE

In [10]:
target = 'duration'
y_train = df_jan[target].values

In [11]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

7.6492610279057605

### Q5. Evaluate on the evaluation set

In [12]:
def wrangle_data(df):
    
    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

df_feb = wrangle_data(df_feb)
df_feb.head()

/tmp/ipykernel_3626/274269193.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.00,0.0,1.0,9.40,2.5,0.00,1.683333
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.00,0.0,1.0,74.65,0.0,1.25,32.083333
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.30,0.0,1.0,25.30,2.5,0.00,13.300000
5,1,2023-02-01 00:52:40,2023-02-01 01:07:18,1.0,5.10,1.0,N,148,236,1,21.9,3.50,0.5,5.35,0.0,1.0,32.25,2.5,0.00,14.633333
6,1,2023-02-01 00:12:39,2023-02-01 00:40:36,1.0,8.90,1.0,N,137,244,1,41.5,3.50,0.5,3.50,0.0,1.0,50.00,2.5,0.00,27.950000


In [13]:
val_dicts = df_feb[categorical].to_dict(orient='records')

X_val = dv.transform(val_dicts)

In [21]:
type(X_val)

scipy.sparse.csr.csr_matrix

In [22]:
type(val_dicts)

list

In [23]:
val_dicts

[{'PULocationID': '142', 'DOLocationID': '163', 'trip_distance': 0.3},
 {'PULocationID': '132', 'DOLocationID': '26', 'trip_distance': 18.8},
 {'PULocationID': '161', 'DOLocationID': '145', 'trip_distance': 3.22},
 {'PULocationID': '148', 'DOLocationID': '236', 'trip_distance': 5.1},
 {'PULocationID': '137', 'DOLocationID': '244', 'trip_distance': 8.9},
 {'PULocationID': '263', 'DOLocationID': '141', 'trip_distance': 1.2},
 {'PULocationID': '48', 'DOLocationID': '243', 'trip_distance': 7.49},
 {'PULocationID': '114', 'DOLocationID': '211', 'trip_distance': 0.51},
 {'PULocationID': '114', 'DOLocationID': '249', 'trip_distance': 0.78},
 {'PULocationID': '125', 'DOLocationID': '107', 'trip_distance': 1.57},
 {'PULocationID': '140', 'DOLocationID': '42', 'trip_distance': 5.4},
 {'PULocationID': '140', 'DOLocationID': '226', 'trip_distance': 3.44},
 {'PULocationID': '249', 'DOLocationID': '90', 'trip_distance': 1.03},
 {'PULocationID': '234', 'DOLocationID': '4', 'trip_distance': 2.06},
 {'

In [14]:
target = 'duration'
y_val = df_feb[target].values

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.81183265470218